<a href="https://colab.research.google.com/github/Maks3174/Phy/blob/main/%D0%94%D0%97_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn


Дані медичне страхування

In [2]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/insurance.csv')

df['sex'] = df['sex'].map({'male': 0, 'female':1})
df['smoker'] = df['smoker'].map({'no': 0, 'yes':1})

df = df.drop(columns=['region'])

data = df[['age', 'bmi', 'expenses']]
df[['age', 'bmi', 'expenses']] = (data - data.min()) / (data.max() - data.min())

In [3]:
df.head()

,age,sex,bmi,children,smoker,expenses
0,0.021739,1,0.320755,0,1,0.251611
1,0.000000,0,0.479784,1,0,0.009636
2,0.217391,0,0.458221,3,0,0.053115
3,0.326087,0,0.180593,0,0,0.333010
4,0.304348,0,0.347709,0,0,0.043816


Потрібно на основі інформації людини спрогнозувати розмір її страховки `expenses`

# Завдання 1
Створіть клас Dataset

In [4]:
class InsuranceDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.X = df.drop(columns=['expenses']).values
        self.y = df['expenses'].values

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        return torch.tensor(self.X[idx], dtype=torch.float32), torch.tensor(self.y[idx], dtype=torch.float32)

# Завдання 2
Створіть DataLoader. Перевірте розміри даних в першому пакеті. Розмір пакетів оберіть самостійно

In [5]:
dataset = InsuranceDataset(df)

batch_size = 32
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

for X_batch, y_batch in dataloader:
    print(f'X_batch shape: {X_batch.shape}')
    print(f'y_batch shape: {y_batch.shape}')
    break

X_batch shape: torch.Size([32, 5])
y_batch shape: torch.Size([32])


# Завдання 3
Створіть нейромережу:
* Кількість шарів оберіть між 1-5
* Виберіть різні функції активації: Relu, LeakyRelu, Swish
* Кілкість нейронів у прихованих шарах повинна **не зростати**

In [10]:
class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class InsuranceModel(nn.Module):
    def __init__(self):
        super(InsuranceModel, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.LeakyReLU(),
            nn.Linear(32, 16),
            Swish(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.network(x)

# Завдання 4
Визначте функцію втрат та оптимізатор

In [11]:
model = InsuranceModel()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Завдання 5
Натренуйте нейромережу. візміть невелику кількість епох

In [12]:
epochs = 10

for epoch in range(epochs):
    for X_batch, y_batch in dataloader:
        y_pred = model(X_batch).squeeze()
        loss = criterion(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

Epoch 1/10, Loss: 0.015689466148614883
Epoch 2/10, Loss: 0.004897157195955515
Epoch 3/10, Loss: 0.009147771634161472
Epoch 4/10, Loss: 0.00833261851221323
Epoch 5/10, Loss: 0.012698527425527573
Epoch 6/10, Loss: 0.004592914134263992
Epoch 7/10, Loss: 0.008374905213713646
Epoch 8/10, Loss: 0.0052056685090065
Epoch 9/10, Loss: 0.003893880872055888
Epoch 10/10, Loss: 0.005100115668028593


# Завдання 6
Збережіть нейромережу та зробіть прогноз

In [13]:
torch.save(model.state_dict(), 'insurance_model.pth')

model = InsuranceModel()
model.load_state_dict(torch.load('insurance_model.pth'))
model.eval()

sample = torch.tensor([0.021739, 1, 0.320755, 0, 1], dtype=torch.float32)
with torch.no_grad():
    prediction = model(sample.unsqueeze(0)).item()

print(f'Predicted insurance expense: {prediction}')

Predicted insurance expense: 0.3300354480743408
